# Build a model with Gensim word embeddings
This will be mostly the same as the other model but instead of having it's own embedding layer it will contain embeddings from Gensim's `Word2Vec`

In [1]:
DATASETS_DIR = "../datasets"
MODELS_DIR = "../models"

In [2]:
# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')

  # read all the text
  text = file.read()

  # close the file
  file.close()
  return text

in_filename = f"{DATASETS_DIR}/content_sequences.txt"
doc = load_doc(in_filename)
lines = doc.split('\n')
lines[:4]

['premier soccer league psl iri kudya magaka mambishi zvichitevera mhirizhonga yakaitika',
 'soccer league psl iri kudya magaka mambishi zvichitevera mhirizhonga yakaitika kubabourfields',
 'league psl iri kudya magaka mambishi zvichitevera mhirizhonga yakaitika kubabourfields kubulawayo',
 'psl iri kudya magaka mambishi zvichitevera mhirizhonga yakaitika kubabourfields kubulawayo nezuro']

## Encode sequences with using `Word2Vec` word index

In [3]:
# define a function to encode text sequences using Word2Vec word index
# instead of Tokenizer's word index
# This is because one of the models will use external word embeddings
def encode_text_with_word2vec(input_text, model, oov_token="<OOV>"):
  # OOV configuration
  oov_index = -1
  oov_token = "<OOV>"

  # list of encoded sequences
  sequences = list()

  for text in input_text:
    # split the text into words (Tokenize it)
    words = text.split()
    sequence = list()

    # map words to indeces using Word2Vec's word index
    for word in words:
      if word in model.wv.key_to_index:
        sequence.append(model.wv.key_to_index[word])
      else:
        # Handle OOV words by mapping them to a special token (oov_token in this case)
        sequence.append(model.wv.key_to_index.get(oov_token,0)) # 0 for unknown words

    sequences.append(sequence)

  return sequences

In [4]:
# load Word2Vec model
from gensim.models import Word2Vec

w2v_model = Word2Vec.load(f"{MODELS_DIR}/word2vec.model")

In [5]:
sequences = encode_text_with_word2vec(lines, w2v_model)

## Sequence inputs and output
Now that there are encoded input sequences, they need to be seperated into input($X$) and output($y$). This can be done via array slicing.

In [6]:
from numpy import array
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [7]:
X.shape

(5498, 10)

In [8]:
y.shape

(5498,)

After seperating, each output word needs to be one hot encoded. This means converting it from an integer to a vector of 0 values, one for each word in the vocabulary, with a 1 to indicate the specific word at the of the words integer value. This is so that the model learns to predict the probability distribution for the next word and the ground truth from which to learn from is 0 for all words except the actual word that comes next.

In [9]:
# one-hot encode the output word
from keras.utils import to_categorical

vocab_size = len(w2v_model.wv.key_to_index)

y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
seq_length

10

## Fit the model: initial training

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional, Dropout
from keras.layers import Embedding
from keras.utils import plot_model

# define the model
def define_model_with_embedding(vocab_size, seq_length, summary=True):
  model = Sequential()
  model.add(Embedding(len(w2v_model.wv.key_to_index), 100, weights=[w2v_model.wv.vectors], trainable=False))
  model.add(Bidirectional(LSTM(100)))
  model.add(Dropout(0.2))
  model.add(Dense(100, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(vocab_size, activation='softmax'))

  # compile network
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # summarize the model
  if summary:
    model.summary()
    model_img_path = f'{MODELS_DIR}/model_with_gensim_embedding.png'
    plot_model(model, to_file=model_img_path, show_shapes=True)
  return model

In [11]:
# define model
model_with_gensim_embedding = define_model_with_embedding(vocab_size, seq_length)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         276500    
                                                                 
 bidirectional (Bidirection  (None, 200)               160800    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 100)               20100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 2765)              279265    
                                                        

In [12]:
# fit model
model_with_gensim_embedding.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
43/43 [==============================] - 24s 66ms/step - loss: 7.8227 - accuracy: 0.0160
Epoch 2/100
43/43 [==============================] - 3s 65ms/step - loss: 7.4956 - accuracy: 0.0238
Epoch 3/100
43/43 [==============================] - 3s 64ms/step - loss: 7.4360 - accuracy: 0.0238
Epoch 4/100
43/43 [==============================] - 3s 66ms/step - loss: 7.4136 - accuracy: 0.0238
Epoch 5/100
43/43 [==============================] - 3s 64ms/step - loss: 7.3982 - accuracy: 0.0238
Epoch 6/100
43/43 [==============================] - 3s 67ms/step - loss: 7.3740 - accuracy: 0.0240
Epoch 7/100
43/43 [==============================] - 3s 70ms/step - loss: 7.3249 - accuracy: 0.0244
Epoch 8/100
43/43 [==============================] - 3s 65ms/step - loss: 7.2385 - accuracy: 0.0256
Epoch 9/100
43/43 [==============================] - 3s 67ms/step - loss: 7.1390 - accuracy: 0.0251
Epoch 10/100
43/43 [==============================] - 3s 71ms/step - loss: 7.0368 - accuracy: 0.025

In [13]:
from pickle import dump

# save the model to file
model_with_gensim_embedding.save(f'{MODELS_DIR}/model_with_gensim_embedding.keras')

In [14]:
# define a function to decoded encoded text sequences produced by the model during
# a prediction
def decode_index_with_word2vec(predicted_word_index, model):
  # OOV configuration
  oov_token = "<OOV>"

  for word, index in model.wv.key_to_index.items():
    if index == predicted_word_index:
      return word

  return oov_token

In [15]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import numpy as np
# generate a sequence from a language model
def generate_seq(model, w2v_model, seq_length, seed_text, n_words):
  result = list()
  in_text = seed_text
  # generate a fixed number of words
  for _ in range(n_words):
    # encode the text as integer
    encoded = encode_text_with_word2vec([in_text], w2v_model)[0]
    # truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # predict probabilities for each word
    yhat = model.predict(encoded, verbose=0)
    # map predicted word index to word
    predicted_word_index = np.argmax(yhat)  # Find the index with the highest probability
    out_word = decode_index_with_word2vec(predicted_word_index, w2v_model)
    # append to input
    in_text += ' ' + out_word
    result.append(out_word)
  return ' '.join(result)

In [18]:
seed_text = lines[randint(0,len(lines))]
pred = generate_seq(model_with_gensim_embedding, w2v_model, seq_length, seed_text, 5)
print(seed_text + ' [START] ' + pred + ' [END]')

fire guard vakakomberedza misha yavo uyewo pove nemafire committee mumasabhuku ose [START] kusvika kuwadhi kuti nekuti zvichibatsira [END]


## Training with cross validation for model selection

In [46]:
from sklearn.model_selection import KFold
import numpy as np

In [47]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
# define 5-fold cross validation test harness
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)

cvscores = []
for train, test in kfold.split(X, y):
    # compile the model
    model_with_embedding = define_model_with_embedding(vocab_size, seq_length, summary=False)
    
    # Fit the model
    model_with_embedding.fit(X[train], y[train], epochs=20, batch_size=128)
    
    # Evaluate the model
    model_with_embedding.evaluate(X[test], y[test])
    print("Accuracy = %s: %.2f%%" % (model_with_embedding.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("Mean Accuracy = %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))